<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/SVM_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 4.2 MB 31.6 MB/s 
     |████████████████████████████████| 1.2 MB 33.1 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 34.6 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0a0cf32ffdd70fc9365121214a7cc7a27f81f33fbf40abcb2bdc391ca100fb57
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [ ]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [ ]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Protests,around workers of Alu Iberica protested in A C...


In [ ]:
#del data_set

In [ ]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
train = model.encode(df.cleaned, device='cuda')

In [ ]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [ ]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[0.12149009, 0.21715306, 0.016726991, -0.62812...",2
1,"[0.08067055, -0.11036323, -0.049742434, -0.210...",2
2,"[0.15647654, 0.23240007, -0.06699539, 0.112573...",0


In [ ]:
 df_features.to_csv("./features_set.csv")

In [ ]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5968198983598922
18.847342217000005


In [ ]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

**Model Training**

SVM

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [ ]:
from sklearn.svm import SVC
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 94.09 seconds


In [ ]:
svm_classifier.score(x_test,y_test)

0.92745

In [ ]:
y_pred = svm_classifier.predict(x_test)

In [ ]:
# y_pred = svm_classifier.predict(x_test)
from sklearn import metrics
# print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.92745


In [ ]:
print(y_pred)
print(y_test)

[0 2 2 ... 1 0 1]
75721    0
80184    2
19864    2
76699    0
92991    5
        ..
32595    4
29313    3
37862    1
53421    0
42410    1
Name: event_type, Length: 20000, dtype: int64


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      4698
           1       0.94      0.93      0.93      2700
           2       0.97      0.99      0.98      6994
           3       0.86      0.81      0.83      1611
           4       0.91      0.82      0.86      1104
           5       0.86      0.91      0.89      2893

    accuracy                           0.93     20000
   macro avg       0.91      0.90      0.90     20000
weighted avg       0.93      0.93      0.93     20000



K FOLD

In [ ]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4294  118    2   32   14  238]
 [ 152 2516    0    5   10   17]
 [   3    0 6890   74   11   16]
 [  30    5  167 1308   23   78]
 [  32   32   29   36  907   68]
 [ 121   18   16   67   37 2634]]


In [ ]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = svm_classifier, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

0.003428374250282506

In [ ]:
# !pip install matplotlib-venn

In [ ]:
# !apt-get -qq install -y libfluidsynth1

In [ ]:
# # https://pypi.python.org/pypi/libarchive
# !apt-get -qq install -y libarchive-dev && pip install -U libarchive
# import libarchive

In [ ]:
# # https://pypi.python.org/pypi/pydot
# !apt-get -qq install -y graphviz && pip install pydot
# import pydot

In [ ]:
# !pip install cartopy
# import cartopy

In [ ]:
# import pandas as pd
# import sklearn
# from sklearn.feature_selection import SelectFromModel
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(
# svm_classifier, x_train, y_train, cv=5, scoring='f1_macro')

In [ ]:
# from sklearn import model_selection
# from sklearn.model_selection import KFold

In [ ]:
# results = model_selection.cross_val_score(svm_classifier,x_train,y_train, cv=KFold)

In [ ]:
# # More algorithms!!!!
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB,BernoulliNB

# rfc = RandomForestClassifier()
# rfc.fit(x_train,y_train)
# print("Score of RFC",rfc.score(x_test,y_test))

# logreg = LogisticRegression()
# logreg.fit(x_train,y_train)
# print("Score of LogReg",logreg.score(x_test,y_test))

# gnb = GaussianNB()
# gnb.fit(x_train,y_train)
# print("Score of GaussianNB",gnb.score(x_test,y_test))

# bnb = BernoulliNB()
# bnb.fit(x_train,y_train)
# print("Score of BernoulliNB",bnb.score(x_test,y_test))